# How to debug tespy models

This notebooks gives some hints on how to use the internal debugging
capabilities of tespy. The user interface of the current implementation might
still need some refinement, and will change based on the feedback in this
session. The outputs shown here are based on the following version:

In [1]:
from tespy import __version__

In [2]:
__version__

'0.9.8.dev0'

## Simple model debugging

This section will show a couple of things

1. How to extract the variables of the problem
  - before presolving step
  - after presolving step and identify the presolved variables
2. How to extract the applied equations of the problem
  - before presolving step
  - after presolving step and identify the presolved equations
3. How to read and fix the errors that are raised during presolving
4. How to debug a model in case of linear dependency by inspecting the
   error message, incidence matrix and Jacobian
5. How to interpret/deal with a couple of warnings/errors that might pop up
   during postprocessing

### Model overview


### Model code

In [3]:
from tespy.components import CycleCloser, SimpleHeatExchanger, Compressor, Valve, Motor, PowerSource
from tespy.connections import Connection, PowerConnection
from tespy.networks import Network

In [4]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    power="kW",
    heat="kW",
    enthalpy="kJ/kg"
)

In [5]:
grid = PowerSource("grid")
motor = Motor("motor")

cc = CycleCloser("cycle closer")
valve = Valve("valve")
compressor = Compressor("compressor")
evaporator = SimpleHeatExchanger("evaporator")
condenser = SimpleHeatExchanger("condenser")

c1 = Connection(cc, "out1", compressor, "in1", label="c1")
c2 = Connection(compressor, "out1", condenser, "in1", label="c2")
c3 = Connection(condenser, "out1", valve, "in1", label="c3")
c4 = Connection(valve, "out1", evaporator, "in1", label="c4")
c5 = Connection(evaporator, "out1", cc, "in1", label="c5")


nw.add_conns(c1, c2, c3, c4, c5)

e1 = PowerConnection(grid, "power", motor, "power_in", label="e1")
e2 = PowerConnection(motor, "power_out", compressor, "power", label="e2")

nw.add_conns(e1, e2)

### Debug the model

#### Variable and equation identification

1. Nothing specified: System has not fluid information, cannot run preproecssing

In [6]:
nw.solve("design", init_only=True)

TESPyNetworkError: The follwing connections of your network are missing any kind of fluid composition information:c1, c2, c3, c4, c5.

2. Fluids specified -> preprocessing runs successfully, but model is obviously
   not working. But we can already extract some information

In [7]:
c1.set_attr(fluid={"R290": 1})
nw.solve("design", init_only=True)
# this would be executed as next step for actual solve call
nw.solve_determination()

You have not provided enough parameters: 10 required, 1 supplied. Aborting calculation!


TESPyNetworkError: You have not provided enough parameters: 10 required, 1 supplied. Aborting calculation!

We can check:

- original variables of the problem
- presolved variables
- actual variables of the problem (representing which original variables)

In [8]:
nw.get_variables_before_presolve()

[('c1', 'm'),
 ('c1', 'p'),
 ('c1', 'h'),
 ('c1', 'fluid'),
 ('c2', 'm'),
 ('c2', 'p'),
 ('c2', 'h'),
 ('c2', 'fluid'),
 ('c3', 'm'),
 ('c3', 'p'),
 ('c3', 'h'),
 ('c3', 'fluid'),
 ('c4', 'm'),
 ('c4', 'p'),
 ('c4', 'h'),
 ('c4', 'fluid'),
 ('c5', 'm'),
 ('c5', 'p'),
 ('c5', 'h'),
 ('c5', 'fluid'),
 ('e1', 'E'),
 ('e2', 'E')]

Presolved variables: fluid composition only in this case

In [9]:
nw.get_presolved_variables()

[('c1', 'fluid'),
 ('c2', 'fluid'),
 ('c3', 'fluid'),
 ('c4', 'fluid'),
 ('c5', 'fluid')]

Actual variables of the problem as a dictionary

- keys: tuple with variable number as first element, variable type as second element
- values: list of variables this variable represents. The list again contains tuples with
  - the label of the component/connection from which the variable originated
  - the type of variable as second element

In [10]:
nw.get_variables()

{(0, 'p'): [('c2', 'p')],
 (1, 'h'): [('c2', 'h')],
 (2, 'p'): [('c3', 'p')],
 (3, 'p'): [('c4', 'p')],
 (4, 'E'): [('e1', 'E')],
 (5, 'E'): [('e2', 'E')],
 (6, 'm'): [('c1', 'm'), ('c2', 'm'), ('c3', 'm'), ('c4', 'm'), ('c5', 'm')],
 (7, 'p'): [('c1', 'p'), ('c5', 'p')],
 (8, 'h'): [('c1', 'h'), ('c5', 'h')],
 (9, 'h'): [('c3', 'h'), ('c4', 'h')]}

There is already a variable mapping: some variables represent more than a
single variable. We can find which equations of the model are responsible for
that

In [11]:
nw.get_presolved_equations()

[('compressor', 'mass_flow_constraints'),
 ('compressor', 'fluid_constraints'),
 ('condenser', 'mass_flow_constraints'),
 ('condenser', 'fluid_constraints'),
 ('cycle closer', 'pressure_equality_constraint'),
 ('cycle closer', 'enthalpy_equality_constraint'),
 ('evaporator', 'mass_flow_constraints'),
 ('evaporator', 'fluid_constraints'),
 ('valve', 'mass_flow_constraints'),
 ('valve', 'fluid_constraints'),
 ('valve', 'enthalpy_constraints')]

Easy way to identify which variable was presolved by which equation?
Not yet...

We can extract the actual equations of our model

In [12]:
nw.get_equations()

{0: ('compressor', ('energy_connector_balance', 0))}

We can also extract the equations of the model with the variables they
depend on

In [13]:
nw.get_equations_with_dependents()

{('compressor', ('energy_connector_balance', 0)): [(1, 'h'),
  (5, 'E'),
  (6, 'm'),
  (8, 'h')]}

3. Specify some parameters and check how the presolved variables change

In [14]:
condenser.set_attr(dp=0)
evaporator.set_attr(dp=0)
compressor.set_attr(eta_s=0.8)
motor.set_attr(eta=0.97)
nw.solve("design", init_only=True)

In [15]:
# number of presolved variables does not change
nw.get_presolved_variables()

[('c1', 'fluid'),
 ('c2', 'fluid'),
 ('c3', 'fluid'),
 ('c4', 'fluid'),
 ('c5', 'fluid')]

In [16]:
# number of presolved equations increases (condenser and evaporator delta p, motor eta)
# compressor eta_s cannot be presolved: only linear dependencies between sets of two variables
nw.get_presolved_equations()

[('compressor', 'mass_flow_constraints'),
 ('compressor', 'fluid_constraints'),
 ('condenser', 'mass_flow_constraints'),
 ('condenser', 'fluid_constraints'),
 ('condenser', 'dp'),
 ('cycle closer', 'pressure_equality_constraint'),
 ('cycle closer', 'enthalpy_equality_constraint'),
 ('evaporator', 'mass_flow_constraints'),
 ('evaporator', 'fluid_constraints'),
 ('evaporator', 'dp'),
 ('motor', 'eta'),
 ('valve', 'mass_flow_constraints'),
 ('valve', 'fluid_constraints'),
 ('valve', 'enthalpy_constraints')]

In [17]:
# number of actual variables reduced by two, because pressures around
# heat exchangers are now respectively represented by a single variable
nw.get_variables()

{(0, 'h'): [('c2', 'h')],
 (1, 'm'): [('c1', 'm'), ('c2', 'm'), ('c3', 'm'), ('c4', 'm'), ('c5', 'm')],
 (2, 'p'): [('c2', 'p'), ('c3', 'p')],
 (3, 'p'): [('c1', 'p'), ('c5', 'p'), ('c4', 'p')],
 (4, 'h'): [('c1', 'h'), ('c5', 'h')],
 (5, 'E'): [('e1', 'E'), ('e2', 'E')],
 (6, 'h'): [('c3', 'h'), ('c4', 'h')]}

In [18]:
# there is now one additional equation: the isentropic efficiency
nw.get_equations_with_dependents()

{('compressor', ('energy_connector_balance', 0)): [(0, 'h'),
  (1, 'm'),
  (4, 'h'),
  (5, 'E')],
 ('compressor', ('eta_s', 0)): [(0, 'h'), (2, 'p'), (3, 'p'), (4, 'h')]}

In [19]:
c1.set_attr(T_dew=10, td_dew=10)  # 10 °C evaporating temperature, 10 K superheating

In [20]:
nw.solve("design", init_only=True)

In [21]:
# number of actual variables reduced by two, because pressure and enthalpy
# can be precalculated based on c1 specifications
nw.get_variables()

{(0, 'h'): [('c2', 'h')],
 (1, 'm'): [('c1', 'm'), ('c2', 'm'), ('c3', 'm'), ('c4', 'm'), ('c5', 'm')],
 (2, 'p'): [('c2', 'p'), ('c3', 'p')],
 (3, 'E'): [('e1', 'E'), ('e2', 'E')],
 (4, 'h'): [('c3', 'h'), ('c4', 'h')]}

In [22]:
# The dependents did change, because now compressor inlet state is known
nw.get_equations_with_dependents()

{('compressor', ('energy_connector_balance', 0)): [(0, 'h'),
  (1, 'm'),
  (3, 'E')],
 ('compressor', ('eta_s', 0)): [(0, 'h'), (2, 'p')]}

Still missing 3 equations as we have 5 variables in the problem and only 2
equations

In [23]:
c3.set_attr(T_bubble=60, td_bubble=0)
e1.set_attr(E=100)  # 100 kW

In [24]:
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.91e+05   | 7 %        | 1.94e-02   | 0.00e+00   | 1.25e+05   | 0.00e+00   | 0.00e+00   
 2     | 2.42e+03   | 29 %       | 3.27e-02   | 0.00e+00   | 1.82e-10   | 0.00e+00   | 0.00e+00   
 3     | 9.33e-10   | 100 %      | 1.37e-14   | 0.00e+00   | 6.37e-11   | 0.00e+00   | 0.00e+00   
 4     | 1.01e-10   | 100 %      | 5.05e-17   | 0.00e+00   | 6.37e-11   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.01 s, Iterations per second: 643.99


#### Errors during presolving

- specify a linear change of specific variable while specifying both values
  simultaneously
- error message directly tells you which variables are linear dependent and
  that you specified more than a single value in that set (points to the labels
  of the connections/components)

In [25]:
e2.set_attr(E=97)
nw.solve("design")

TESPyNetworkError: You specified more than one variable of the linear dependent variables: (e1: E), (e2: E).

- same problem if you were to specify compressor pressure ratio, as both inlet
  and outlet pressure are already determined and connected via the delta p of
  the condenser
- Which variables you specified is not (yet) pointed at

In [26]:
e2.set_attr(E=None)
compressor.set_attr(pr=4)
nw.solve("design")

TESPyNetworkError: You specified more than one variable of the linear dependent variables: (c1: p), (c2: p), (c5: p), (c4: p), (c3: p).

- create a circular dependency, this error is raised before duplicated
  specification
- the output is the variables which are part of the circular dependency and
  the equations responsible for that (-> so why not before?! TODO for meeting
  hackathon)


In [27]:
valve.set_attr(pr=1/4)
nw.solve("design")

TESPyNetworkError: A circular dependency between the variables ('c1', 'p'), ('c2', 'p'), ('c3', 'p'), ('c4', 'p'), ('c5', 'p') caused by the equations ('compressor', 'pr'), ('condenser', 'dp'), ('cycle closer', 'pressure_equality_constraint'), ('evaporator', 'dp'), ('valve', 'pr') has been detected. This overdetermines the problem.

- Specify three different parameters for fluid properties on single connection
- Gives the specifications and the connection label

In [28]:
c1.set_attr(p=10)  # p has already been set!
valve.set_attr(pr=None)
compressor.set_attr(pr=None)
nw.solve("design")

TESPyNetworkError: You have specified more than 2 parameters for the connection c1 with a known fluid composition: p, T_dew, td_dew. This overdetermines the state of the fluid.

In [29]:
c1.set_attr(p=None)

#### Linear dependency inspection

- construct a linear dependency:
  - fix heat output of condenser (we have fixed motor electrical power already)
  - no specification of evaporator delta p

In [30]:
condenser.set_attr(Q=-350)
evaporator.set_attr(dp=None)
nw.solve("design")

Detected singularity in Jacobian matrix. This singularity is most likely caused by the parametrization of your problem and NOT a numerical issue. Double check your setup.
The following variables of your problem are not in connection with any equation: (0, 'p')




 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.67e+04   | 13 %       | NaN        | NaN        | NaN        | NaN        | NaN        
Total iterations: 1, Calculation time: 0.00 s, Iterations per second: 1037.42


- solver tells us
  - setup problem, not a numerical issue
  - variable (0, "p") is not associated with any equation
- retrieve variables to find, what that is pointing to -> c4 pressure (evaporator inlet)

In [31]:
nw.get_variables()

{(0, 'p'): [('c4', 'p')],
 (1, 'h'): [('c2', 'h')],
 (2, 'm'): [('c1', 'm'), ('c2', 'm'), ('c3', 'm'), ('c4', 'm'), ('c5', 'm')]}

- for other types of errors, we will need a different setup (equations are too simple here)
- just a heat exchanger is sufficient

In [32]:
from tespy.components import Source, Sink, MovingBoundaryHeatExchanger

In [33]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar"
)

In [34]:
so1 = Source("source 1")
so2 = Source("source 2")

si1 = Sink("sink 1")
si2 = Sink("sink 2")

heatex = MovingBoundaryHeatExchanger("heatexchanger")

c1 = Connection(so1, "out1", heatex, "in1", label="c1")
c2 = Connection(heatex, "out1", si1, "in1", label="c2")
d1 = Connection(so2, "out1", heatex, "in2", label="d1")
d2 = Connection(heatex, "out2", si2, "in1", label="d2")

nw.add_conns(c1, c2, d1, d2)

- we make a specification that is impossible, minimum terminal temperature
  difference of 25 K but at the same time fixing temperature at hot side outlet
  and cold side inlet (leading to a temperature difference of 20)

In [35]:
c1.set_attr(fluid={"air": 1}, T=200, p=1, m=5)
c2.set_attr(T=110)
d1.set_attr(fluid={"water": 1}, T=90, p=1)
heatex.set_attr(dp1=0, dp2=0, ttd_min=25)

In [36]:
nw.solve("design")

Found singularity in Jacobian matrix, calculation aborted! The setup of you problem seems to be solvable. It failed due to partial derivatives in the Jacobian being zero, which were expected not to be zero, or the other way around. The reason for this usually lies in starting value selection or bad convergence. The following equations (key of outer dict) may have an unexpected zero/non-zero in the partial derivative towards the variable (value of outer dict) and be the root of evil: {1: ('heatexchanger', ('ttd_min', 0))}: {(0, 'h'): [('d2', 'h')]}The following equations of your problem do not depend on any variable: ('heatexchanger', ('ttd_min', 0))




 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.20e+06   | 0 %        | 1.15e+00   | 0.00e+00   | 1.48e+05   | 0.00e+00   | 0.00e+00   
 2     | 1.71e+05   | 8 %        | NaN        | NaN        | NaN        | NaN        | NaN        
Total iterations: 2, Calculation time: 0.01 s, Iterations per second: 295.03


- solver tells us potentially numerical issue (not always correct!)
- the heat exchanger ttd_min function should depend on variable (0, "h")
  representing "h" of connection "d2"
- there is a zero in the Jacobian of that function towards that number while
  there should be a non-zero entry (or there is a non-zero entry while a zero
  entry is expected)
- extra piece of information: all entries in the Jacobian of the equation are
  zero (they never should be!!)
- extract the incidence matrix and the Jacobian to identify issue if we want

In [37]:
nw.get_equations_with_dependents()

{('heatexchanger', ('energy_balance_constraints', 0)): [(0, 'h'), (1, 'm')],
 ('heatexchanger', ('ttd_min', 0)): [(0, 'h')]}

- equation number is 1
- variable number is 0

- in the incidence matrix it is indicated which entries (rows = equations,
  columns = variables) are supposed to be zero or non-zero
- in the Jacobian the actual values are stored
- see that a non-zero is expected for (1, 0) but there is a zero instead

In [38]:
nw._incidence_matrix_dense

array([[1., 1.],
       [1., 0.]])

In [39]:
nw.jacobian

array([[2.56784202e-01, 2.44827106e+06],
       [0.00000000e+00, 0.00000000e+00]])

#### Postprocessing warnings/errors

- typical problem with kA/UA specification and approaching temperatures
- using flat temperature on water side (just evaporating fluid)

In [40]:
heatex.set_attr(ttd_min=None)
d1.set_attr(x=0, T=None)
d2.set_attr(x=1)
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 2.73e+06   | 0 %        | 1.21e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 2     | 5.72e-07   | 100 %      | 2.53e-13   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 3     | 5.82e-11   | 100 %      | 2.58e-17   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
 4     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 2916.26


- implement a version, where the air outlet temperature is obtained from kA
- the larger kA, the smaller the gap between the air outlet temperature and
  the water evaporation temperature
- calculation converges due to internal convergence helpers, but result is not
  correct

In [ ]:
heatex.set_attr(kA=heatex.kA.val * 10)  # this should pin the air outlet temperature very close to the evaporation temperature
c2.set_attr(T=None)
nw.solve("design")

Invalid value for ttd_l: ttd_l = -148.35112323967502 below minimum value (0) at component heatexchanger.


Invalid value for ttd_min: ttd_min = -148.35112323967502 below minimum value (0) at component heatexchanger.


Invalid value for eff_hot: eff_hot = 2.4626071825331164 above maximum value (1) at component heatexchanger.


Invalid value for eff_max: eff_max = 2.4626071825331164 above maximum value (1) at component heatexchanger.


Invalid value for td_pinch: td_pinch = -148.35112323967502 below minimum value (0) at component heatexchanger.


The simulation converged but the calculated result nan watt / kelvin for the fixed input parameter kA is not equal to the originally specified value: 115443.49187139163. Usually, this can happen, when a method internally manipulates the associated equation during iteration in order to allow progress in situations, when the equation is otherwise not well defined for the currentvalues of the variables, e.g. in case a negative root would need to be evaluated.  Often, this can happen during the first iterations and then will resolve itself as convergence progresses. In this case it did not, meaning convergence was not actually achieved.



 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 4.12e+06   | 0 %        | 6.22e-02   | 0.00e+00   | 2.81e+04   | 0.00e+00   | 0.00e+00   
 2     | 6.59e+05   | 2 %        | 2.92e-01   | 0.00e+00   | 1.32e+05   | 0.00e+00   | 0.00e+00   
 3     | 9.98e-03   | 88 %       | 1.41e-14   | 0.00e+00   | 1.41e-03   | 0.00e+00   | 0.00e+00   
 4     | 5.21e-10   | 100 %      | 1.03e-16   | 0.00e+00   | 4.66e-11   | 0.00e+00   | 0.00e+00   
 5     | 0.00e+00   | 100 %      | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   | 0.00e+00   
Total iterations: 5, Calculation time: 0.01 s, Iterations per second: 835.69


In [42]:
nw.status

2